In [ ]:
# https://machinelearningmastery.com/how-to-use-transfer-learning-when-developing-convolutional-neural-network-models/
# Classifier: The pre-trained model is used directly to classify new images.
# Weight Initialization: The pre-trained model, or some portion of the model, is integrated into a new model, and the layers of the pre-trained model are trained in concert with the new model.

In [ ]:
import numpy as np
import math
import os
import random
import cv2
import matplotlib.pyplot as plt
from keras.utils import to_categorical

from keras import Input, regularizers, Model
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet import ResNet50
from keras.applications.xception import Xception
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, Dropout, BatchNormalization, GlobalAveragePooling2D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive

'11 Sep 2020 - ₹ 1,500 Paid to TRAINING ACCOUNT PLACEMENT OFFICE TRAINING, PLACEMENT AND PUBLIC RELATIONS on Google Pay [Paid ₹ 1,500].jpg'
 A2PartC_vdeo_frames
 ADSA_END_SEM.pdf
 Classroom
'Colab Notebooks'
 CS20M045_Piyush.pdf
 CS20M045_Piyush_System.pdf
 demo_augumentation
 exam69.gdoc
 exam69.pdf
 example_report4-converted.docx
'First APL Problem.gdoc'
 GSO5F77R8D9DPLV9.mp4
'Meet Recordings'
 me.jpg
 ML_Tutorial
 model.h5
'MTech Schedule.gdoc'
 Myntra_Project_1.pptx
 Myntra_Project_2.pptx
 MyntraProject.drawio
 nature_12K
'nature_12K (1)'
 nature_12k_new
'New doc 17-Mar-2021 10.58 PM.jpg'
 Resume_all.gdoc
 Resume_for_System.gdoc
 Resume_new.gdoc
 Resume_Template_all.gdoc
 Resume_Template_all.pdf
 sampleVideoImages
 TAO-Problem-solving.gdoc
'Zombieland problem with merge sort approach.gdoc'


In [ ]:
dir_train = "/content/drive/MyDrive/demo_augumentation"
# dir_train = "/content/drive/MyDrive/nature_12k_new/inaturalist_12K/train"
dir_val = "/content/drive/MyDrive/nature_12k_new/inaturalist_12K/val"
class_list = os.listdir(dir_train)
if '.DS_Store' in class_list:
  class_list.remove('.DS_Store')
class_list

['Reptilia',
 'Mollusca',
 'Animalia',
 'Amphibia',
 'Aves',
 'Mammalia',
 'Insecta',
 'Plantae',
 'Arachnida',
 'Fungi']

In [ ]:
num_per_cls = 100

In [ ]:
x_amp_train = []
x_amp_test = []

y_amp_train = []
y_amp_test = []

for i in range(len(class_list)):
  dir = dir_train+"/"+class_list[i]
  img_name = random.sample(os.listdir(dir), k=(num_per_cls+math.ceil(0.1*num_per_cls)))
  for inc in range(num_per_cls):
    # print(inc)
    # print(img_name[inc])
    img = cv2.resize(cv2.imread(os.path.join(dir, img_name[inc])), (350,350))
    # print(type(img))
    if img is not None:
        x_amp_train.append(img)
        y_amp_train.append(i)

  print("----------------------------------------")
  for inc in range(num_per_cls, num_per_cls+math.ceil(0.1*num_per_cls)):
    # print(inc)
    # print(img_name[inc])
    img = cv2.resize(cv2.imread(os.path.join(dir, img_name[inc])), (350,350))
    if img is not None:
        x_amp_test.append(img)
        y_amp_test.append(i)
  print(len(x_amp_train), len(x_amp_test))

----------------------------------------
100 10
----------------------------------------
200 20
----------------------------------------
300 30
----------------------------------------
400 40
----------------------------------------
500 50
----------------------------------------
600 60
----------------------------------------
700 70
----------------------------------------
800 80
----------------------------------------
900 90
----------------------------------------
1000 100


In [ ]:
y_amp_train = to_categorical(y_amp_train)
y_amp_test = to_categorical(y_amp_test)

In [ ]:
print(len(x_amp_train),len(x_amp_test))

1000 100


In [ ]:
x_amp_train = np.array(x_amp_train)
x_amp_test = np.array(x_amp_test) 

In [ ]:
y_amp_train.shape

(1000, 10)

In [ ]:
# load model
model = Xception()
model.summary()
len(model.layers)

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

134

In [ ]:
# load model
new_input = Input(shape=(350, 350, 3))
model = Xception(weights='imagenet',include_top=False , input_tensor=new_input)
x = (GlobalAveragePooling2D())(model.layers[-1].output)
output = (Dense(10, activation="softmax", kernel_regularizer=regularizers.l2(0.01)))(x)
model = Model(inputs=model.inputs, outputs=output)
# summarize the model
model.summary()
len(model.layers)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 350, 350, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 174, 174, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 174, 174, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 174, 174, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

134

In [ ]:
model.layers[3]

In [ ]:
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], options = run_opts)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# If ResourceExhaustedError occurs and If you're using a pre-trained model - Solution
# 1. Reduce Image Dimension
# 2. Reduce Batchsize - 32 -> 8/16/24
# 3. freeze the first layers by https://keras.io/getting_started/faq/#how-can-i-freeze-layers-and-do-finetuning - model.layers[i].trainable = False
len(model. layers)
# for i in range(0, 4):
#   model.layers[i].trainable = False
#   assert model.layers[i].trainable_weights == []
#   # assert len(model.trainable_weights) == 2

history = model.fit(x_amp_train, y_amp_train, validation_data=(x_amp_test, y_amp_test), epochs=10, batch_size=16)

Epoch 1/10
63/63 [==============================] - 112s 2s/step - loss: 1.9210 - accuracy: 0.4180 - val_loss: 14.7067 - val_accuracy: 0.1000
Epoch 2/10
63/63 [==============================] - 106s 2s/step - loss: 1.3078 - accuracy: 0.6250 - val_loss: 2.3020 - val_accuracy: 0.3400
Epoch 3/10
63/63 [==============================] - 106s 2s/step - loss: 0.8694 - accuracy: 0.7670 - val_loss: 2.1534 - val_accuracy: 0.4600
Epoch 4/10
63/63 [==============================] - 106s 2s/step - loss: 0.6888 - accuracy: 0.8180 - val_loss: 2.1416 - val_accuracy: 0.3800
Epoch 5/10
63/63 [==============================] - 106s 2s/step - loss: 0.5321 - accuracy: 0.8550 - val_loss: 2.1217 - val_accuracy: 0.4900
Epoch 6/10
63/63 [==============================] - 106s 2s/step - loss: 0.4406 - accuracy: 0.8840 - val_loss: 2.0033 - val_accuracy: 0.5300
Epoch 7/10
63/63 [==============================] - 106s 2s/step - loss: 0.3217 - accuracy: 0.9290 - val_loss: 2.4856 - val_accuracy: 0.4800
Epoch 8/10
6

In [ ]:
accuracy, loss, val_accuracy, val_loss = history.history['accuracy'], history.history['loss'], history.history['val_accuracy'], history.history['val_loss']
print(accuracy)

[0.4180000126361847, 0.625, 0.7670000195503235, 0.8180000185966492, 0.8550000190734863, 0.8840000033378601, 0.9290000200271606, 0.8999999761581421, 0.9259999990463257, 0.9490000009536743]


In [ ]:
# model = VGG16(weights='imagenet',include_top=False , input_tensor=new_input, pooling='max')
# layer = (BatchNormalization())(model.layers[0].output)
# layer = (model.layers[1].output)(layer)
# output = Dense(10, activation="softmax", kernel_regularizer=regularizers.l2(0.01))(block1_conv1_bn)
# model = Model(inputs=model.inputs, outputs=output)
# model.summary()

In [ ]:
# # By default init weights of imagenet
# # include_top = False will discard classification layer of model
# # classes = 10 will take output to classify 10 classes
# # pooling has choices max and avg
# # new_input = Input(shape=(500, 500, 3))
# new_input = Input(shape=(350, 350, 3))
# #model = VGG16(weights='imagenet',include_top=False , input_tensor=new_input, classes=10, pooling='max', classifier_activation='softmax')
# model = VGG16(weights='imagenet',include_top=False , input_tensor=new_input, pooling='max')
# # add new classifier layers
# flat1 = Flatten()(model.layers[-1].output) # .output
# class1 = Dense(4096, activation='relu')(flat1)
# class2 = Dense(4096, activation='relu')(class1)
# output = Dense(10, activation="softmax", kernel_regularizer=regularizers.l2(0.01))(class2)
# # define new model
# model = Model(inputs=model.inputs, outputs=output)
# # summarize
# model.summary()

In [ ]:
# model.layers[-1].output

In [ ]:
# tf.compat.v1.RunOptions

In [ ]:
# import tensorflow as tf
# run_opts = tf.compat.v1.RunOptions(report_tensor_allocations_upon_oom = True)

In [ ]:
# Parameters
# epochs
# weight decay
# pooling